<a href="https://colab.research.google.com/github/RockhoRockho/Data-project/blob/main/19_RNN_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from glob import glob
from tqdm import tqdm

In [ ]:
LABEL_INT_DICT = np.unique(pd.read_csv('drive/Mydrive/ucf11_train_vid.csv')['label'])
LABEL_STR_DICT = {k:v for v, k in enumerate(LABEL_INT_DICT)}
pprint(LABEL_INT_DICT)
pprint(LABEL_STR_DICT)

FileNotFoundError: ignored

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

MAX_FRAMES = 10
NUM_FEATURES = 1280

def build_model():
  inputs = keras.Input((MAX_FRAMES, NUM_FEATURES))
  x = keras.layers.LSTM(64, return_sequences=True)( # 받은 데이터를 그대로 넘기는 것(input 유지)
      inputs
  )
  x = keras.layers.LSTM(64, return_sequences=False)(
      x
  )
  x = keras.layers.Dropout(0.3)(x)
  outputs = keras.layers.Dense(len(LABEL_INT_DICT), activation='softmax')(x)
  model = keras.Model(inputs, outputs)
  return model

adam = keras.optimizers.Adam(lr=0.0001)
model = build_model()

model.compile(
    optimizer=adam,
    loss = 'sparse_categorical_crossentropy',
    metrics = 'accuracy'
)

model.summary()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10, 1280)]        0         
                                                                 
 lstm_2 (LSTM)               (None, 10, 64)            344320    
                                                                 
 lstm_3 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 11)                715       
                                                                 
Total params: 378,059
Trainable params: 378,059
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import albumentations as A
import cv2
import math
import tensorflow as tf
from tensorflow import keras

class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size,
        mode='train', shuffle=True):
        assert mode in ['train', 'valid']
        self.mode = mode
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.npy_paths = glob(
            f'drive/MyDrive/UCF11_updated_npy/{mode}/*.npy'
        )
    
    def __len__(self):
        return math.ceil(len(self.npy_paths) / self.batch_size)
    
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.npy_paths[strt:fin]
    
        batch_x, batch_y = self.get_data(data)
        return np.array(batch_x), np.array(batch_y)
    
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        for npy_path in data:
            npy = np.load(npy_path)
            
            label = npy_path.split('/')[-1].split('_')[0]
            label = LABEL_STR_DICT[label]
            
            batch_x.append(npy)
            batch_y.append(label)
            
        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.npy_paths)
    
train_generator = DataGenerator(
    mode='train',
    batch_size=128,
    shuffle=True)

valid_generator = DataGenerator(
    mode='valid',
    batch_size=128,
    shuffle=True)

In [ ]:
for x, y in train_generator:
  print(x.shape)
  print(y.shape)
  break

(128, 10, 1280)
(128,)


In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=30,
    verbose=1
)

Epoch 1/30
2/2 [==============================] - 18s 14s/step - loss: 2.3855 - accuracy: 0.1000 - val_loss: 2.2874 - val_accuracy: 0.3273
Epoch 2/30
2/2 [==============================] - 0s 261ms/step - loss: 2.3313 - accuracy: 0.1500 - val_loss: 2.2493 - val_accuracy: 0.3636
Epoch 3/30
2/2 [==============================] - 0s 218ms/step - loss: 2.2829 - accuracy: 0.2091 - val_loss: 2.2103 - val_accuracy: 0.4364
Epoch 4/30
2/2 [==============================] - 0s 223ms/step - loss: 2.2441 - accuracy: 0.2773 - val_loss: 2.1720 - val_accuracy: 0.4364
Epoch 5/30
2/2 [==============================] - 0s 291ms/step - loss: 2.2054 - accuracy: 0.3455 - val_loss: 2.1341 - val_accuracy: 0.4909
Epoch 6/30
2/2 [==============================] - 0s 265ms/step - loss: 2.1504 - accuracy: 0.4364 - val_loss: 2.0962 - val_accuracy: 0.5273
Epoch 7/30
2/2 [==============================] - 0s 230ms/step - loss: 2.1140 - accuracy: 0.4864 - val_loss: 2.0587 - val_accuracy: 0.5636
Epoch 8/30
2/2 [=====

In [ ]:
import matplotlib.pyplot as plt
history = history.history

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("Loss")

plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='train')
plt.plot(history['val_accuracy'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title("Accuracy")
plt.show()

AttributeError: ignored